In [ ]:
import torch
import torch.nn as nn

In [ ]:
# Adapted from pytorch-ident, https://github.com/forgi86/pytorch-ident/blob/master/torchid/ss/dt/models.py
class NeuralStateUpdate(nn.Module):

    def __init__(self, n_x=2, n_u=1, n_feat=32):
        super(NeuralStateUpdate, self).__init__()
        
        self.net = nn.Sequential(
            nn.Linear(n_x+n_u, n_feat),
            nn.Tanh(),
            nn.Linear(n_feat, n_x),
        )
        
        for m in self.net.modules():
            if isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, mean=0, std=1e-2)
                nn.init.constant_(m.bias, val=0)


    def forward(self, x, u):
        z = torch.cat((x, u), dim=-1)
        dx = self.net(z)
        return dx

In [ ]:
class NeuralOutput(nn.Module):

    def __init__(self, n_x, n_y, n_feat=32):
        super(NeuralOutput, self).__init__()
        
        self.net = nn.Sequential(
            nn.Linear(n_x, n_feat),
            nn.Tanh(),
            nn.Linear(n_feat, n_y),
        )
                    
    def forward(self, x):
        y = self.net(x)
        return y

In [ ]:
class StateSpaceSimulator(nn.Module):
    def __init__(self, f_xu):
        super().__init__()
        self.f_xu = f_xu

    def forward(self, x_0, u):
        B, n_x = x_0.shape
        _, T, _ = u.shape # B, T, n_u
        x = torch.empty((B, T, n_x))
        x_step = x_0

        # manually unroll f_xu over time
        for t in range(T): 
            x[:, t, :] = x_step
            dx = self.f_xu(x_step, u[:, t, :])
            x_step = x_step + dx

        return x

## Basic usage

In [ ]:
n_x = 2; n_u = 1;
f_xu = NeuralStateUpdate(n_x, n_u, n_feat=32)
simulator = StateSpaceSimulator(f_xu)

In [ ]:
B, T = 32, 1024; 
batch_x0 = torch.zeros((B, n_x))
batch_u = torch.randn((B, T, n_u)) # replace with actual training input
batch_x_sim = simulator(batch_x0, batch_u) # B, T, n_x 
batch_x_sim.shape

In [ ]:
# Overall, we have defined a sort of custom RNN. In fact, it behaves pretty much like a standard LSTM
lstm = torch.nn.RNN(input_size=n_u, hidden_size=n_x, batch_first=True, num_layers=1)
batch_h, _ = lstm(batch_u)
batch_h.shape

In [ ]:
#batch_h = lstm(batch_u, hx=batch_x0[None, ...]) # hx has format (num_layers=1, batch_size, hidden_size)

## For Cascaded Two-tanks

In [ ]:
n_y = 1; n_x = 2; n_u = 1;
B = 1 # just one sequence
T = 1024
u = torch.randn((B, T, n_u)) # replace with actual training input
y = torch.randn((B, T, n_y)) # replace with actual training output

In [ ]:
x0 = torch.zeros((B, n_x), requires_grad=True) # this is also a training variable
f_xu = NeuralStateUpdate(n_x, n_u, n_feat=32)
g_x = NeuralOutput(n_x, n_y)
simulator = StateSpaceSimulator(f_xu) # 

In [ ]:
opt = torch.optim.AdamW([
        {'params': f_xu.parameters(),    'lr': 1e-3},
        {'params': g_x.parameters(),    'lr': 1e-3},
        {'params': x0 , 'lr': 1e-3},
    ], 1e-3
)
#opt = torch.optim.AdamW(list(f_xu.parameters()) + list(g_x.parameters()) + [x0], 1e-3)

In [ ]:

x_sim = simulator(x0, u) # B, T, n_x
y_sim = g_x(x_sim) # # B, T, n_y
loss = torch.nn.functional.mse_loss(y, y_sim)